In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt

In [43]:
all_year_ave = pd.DataFrame()

In [6]:
def calculate_day_of_year(row):
    date = row['Date']
    year = row['Year']

    is_leap_year = year % 4 == 0

    day_of_year = date.dayofyear
    
    if is_leap_year and date.month > 2:
        day_of_year -= 1
    
    return int(day_of_year)

In [55]:
originalCSV = "sanfrancisco.csv"
modifiedPath = "sf_post1980.csv"


In [56]:
key ='Mean.TemperatureF'
year = 1980
df = pd.read_csv(originalCSV, usecols=['Date', key,'season','Mean.Humidity'])
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df

,Date,Mean.TemperatureF,Mean.Humidity,season,Year
0,1948-01-01,50.0,87.0,Winter,1948
1,1948-01-02,55.0,91.0,Winter,1948
2,1948-01-03,54.0,94.0,Winter,1948
3,1948-01-04,57.0,91.0,Winter,1948
4,1948-01-05,55.0,91.0,Winter,1948
...,...,...,...,...,...
24738,2015-12-27,41.0,71.0,Winter,2015
24739,2015-12-28,45.0,77.0,Winter,2015
24740,2015-12-29,45.0,67.0,Winter,2015
24741,2015-12-30,45.0,80.0,Winter,2015


In [57]:
filtered_df = df[df['Date'].dt.year >= year]
filtered_df = filtered_df[~((filtered_df['Date'].dt.month == 2) & (filtered_df['Date'].dt.day == 29))]
filtered_df = filtered_df.reset_index(drop=True)

for index, row in filtered_df.iterrows():
    filtered_df.loc[index, 'Day'] = calculate_day_of_year(row)
    if (calculate_day_of_year(row)> 365):
        print(row)

filtered_df['Day'] = filtered_df['Day'].astype(int)
filtered_df

,Date,Mean.TemperatureF,Mean.Humidity,season,Year,Day
0,1980-01-01,50.0,93.0,Winter,1980,1
1,1980-01-02,44.0,98.0,Winter,1980,2
2,1980-01-03,50.0,84.0,Winter,1980,3
3,1980-01-04,50.0,83.0,Winter,1980,4
4,1980-01-05,50.0,79.0,Winter,1980,5
...,...,...,...,...,...,...
13042,2015-12-27,41.0,71.0,Winter,2015,361
13043,2015-12-28,45.0,77.0,Winter,2015,362
13044,2015-12-29,45.0,67.0,Winter,2015,363
13045,2015-12-30,45.0,80.0,Winter,2015,364


In [58]:
df_2001 = df[df['Date'].dt.year == 2001].copy()
df_1999 = df[df['Date'].dt.year == 1999].copy()

In [59]:
date_range = pd.date_range(start="2000-01-01", end="2000-12-31", freq='D')
year2000 = filtered_df[filtered_df['Date'].dt.year == 2000]
missing_dates = date_range.difference(year2000['Date'])
missing_df = pd.DataFrame()


for missing_date in missing_dates:
    if missing_date.month == 2 and missing_date.day == 29:
        continue
    date_in_1999 = missing_date.replace(year=1999)
    date_in_2001 = missing_date.replace(year=2001)
    
    row_1999 = df_1999[df_1999['Date'] == date_in_1999]
    row_2001 = df_2001[df_2001['Date'] == date_in_2001]
    
    if not row_1999.empty and not row_2001.empty:
        avg_temp = (row_1999['Mean.TemperatureF'].values[0] + row_2001['Mean.TemperatureF'].values[0]) / 2
        avg_humidity = (row_1999['Mean.Humidity'].values[0] + row_2001['Mean.Humidity'].values[0]) / 2
        
        day_of_year = missing_date.dayofyear
        if missing_date.month > 2:
            day_of_year -= 1
        missing_row = pd.DataFrame({
            'Date': [missing_date],
            'Mean.TemperatureF': [avg_temp],
            'Mean.Humidity': [avg_humidity],
            'season': [row_1999['season'].values[0]], 
            'Year': [2000],
            'Day': [day_of_year]
        })
        
        missing_df = pd.concat([missing_df, missing_row], ignore_index=True)

missing_df

,Date,Mean.TemperatureF,Mean.Humidity,season,Year,Day
0,2000-02-23,50.0,84.0,Winter,2000,54
1,2000-02-24,50.0,88.5,Winter,2000,55
2,2000-02-25,51.0,79.5,Winter,2000,56
3,2000-02-26,49.5,80.5,Winter,2000,57
4,2000-02-27,53.5,69.0,Winter,2000,58
...,...,...,...,...,...,...
88,2000-08-24,62.0,84.0,Summer,2000,236
89,2000-08-25,64.0,77.0,Summer,2000,237
90,2000-08-28,63.0,79.5,Summer,2000,240
91,2000-08-29,62.5,81.5,Summer,2000,241


In [60]:

df_2000_complete = pd.concat([filtered_df, missing_df], ignore_index=True)
df_2000_complete = df_2000_complete.sort_values('Date').reset_index(drop=True)
df_2000_complete

,Date,Mean.TemperatureF,Mean.Humidity,season,Year,Day
0,1980-01-01,50.0,93.0,Winter,1980,1
1,1980-01-02,44.0,98.0,Winter,1980,2
2,1980-01-03,50.0,84.0,Winter,1980,3
3,1980-01-04,50.0,83.0,Winter,1980,4
4,1980-01-05,50.0,79.0,Winter,1980,5
...,...,...,...,...,...,...
13135,2015-12-27,41.0,71.0,Winter,2015,361
13136,2015-12-28,45.0,77.0,Winter,2015,362
13137,2015-12-29,45.0,67.0,Winter,2015,363
13138,2015-12-30,45.0,80.0,Winter,2015,364


In [61]:
for index, row in df_2000_complete.iterrows():
    if (pd.isna(row['Mean.TemperatureF']) or pd.isnull(row['Mean.TemperatureF'])):
        date_in_1999 = row['Date'].replace(year=1999)
        date_in_2001 = row['Date'].replace(year=2001)
        
        row_1999 = df_1999[df_1999['Date'] == date_in_1999]
        row_2001 = df_2001[df_2001['Date'] == date_in_2001]
        df_2000_complete.at[index, 'Mean.TemperatureF'] = (
                row_1999['Mean.TemperatureF'].values[0] + row_2001['Mean.TemperatureF'].values[0]
            ) / 2
    if (pd.isna(row['Mean.Humidity']) or pd.isnull(row['Mean.Humidity'])):
        date_in_1999 = row['Date'].replace(year=1999)
        date_in_2001 = row['Date'].replace(year=2001)
        
        row_1999 = df_1999[df_1999['Date'] == date_in_1999]
        row_2001 = df_2001[df_2001['Date'] == date_in_2001]
        df_2000_complete.at[index, 'Mean.Humidity'] = (
                row_1999['Mean.Humidity'].values[0] + row_2001['Mean.Humidity'].values[0]
            ) / 2

In [39]:
df_2000_complete

,Date,Mean.TemperatureF,Mean.Humidity,season,Year,Day
0,1980-01-01,62.0,73.0,Winter,1980,1
1,1980-01-02,56.0,67.0,Winter,1980,2
2,1980-01-03,62.0,61.0,Winter,1980,3
3,1980-01-04,70.0,83.0,Winter,1980,4
4,1980-01-05,60.0,68.0,Winter,1980,5
...,...,...,...,...,...,...
13135,2015-12-27,80.0,75.0,Winter,2015,361
13136,2015-12-28,80.0,78.0,Winter,2015,362
13137,2015-12-29,80.0,77.0,Winter,2015,363
13138,2015-12-30,81.0,81.0,Winter,2015,364


In [16]:
df_2000_complete.to_csv(modifiedPath, index=False)

In [62]:
df_2000_complete['Year'] = df_2000_complete['Date'].dt.year
df_2000_complete['Month'] = df_2000_complete['Date'].dt.month

monthly_stats = df_2000_complete.groupby(['Year', 'Month']).agg(
    Max_Temperature=('Mean.TemperatureF', 'max'),
    Min_Temperature=('Mean.TemperatureF', 'min'),
    Mean_Temperature=('Mean.TemperatureF', 'mean'),
    Max_Humidity=('Mean.Humidity', 'max'),
    Min_Humidity=('Mean.Humidity', 'min'),
    Mean_Humidity=('Mean.Humidity', 'mean')
).reset_index()

monthly_stats

,Year,Month,Max_Temperature,Min_Temperature,Mean_Temperature,Max_Humidity,Min_Humidity,Mean_Humidity
0,1980,1,64.0,44.0,50.258065,98.0,53.0,77.225806
1,1980,2,60.0,48.0,54.392857,86.0,39.0,75.107143
2,1980,3,57.0,48.0,52.709677,83.0,52.0,68.387097
3,1980,4,64.0,50.0,55.366667,81.0,41.0,70.700000
4,1980,5,60.0,51.0,55.838710,84.0,60.0,72.193548
...,...,...,...,...,...,...,...,...
427,2015,8,79.0,65.0,68.258065,78.0,56.0,69.774194
428,2015,9,77.0,62.0,67.333333,80.0,41.0,64.400000
429,2015,10,75.0,62.0,65.741935,83.0,54.0,69.967742
430,2015,11,64.0,45.0,54.600000,80.0,46.0,67.066667


In [64]:
all_years_stats = monthly_stats.groupby('Month').agg({
    'Max_Temperature': 'max',   
    'Min_Temperature': 'min',   
    'Mean_Temperature': 'mean', 
    'Max_Humidity': 'max',      
    'Min_Humidity': 'min',      
    'Mean_Humidity': 'mean'    
}).reset_index()


all_years_stats.columns = [
    'Month',
    'Max_Temperature_All_Years', 
    'Min_Temperature_All_Years', 
    'Mean_Temperature_All_Years', 
    'Max_Humidity_All_Years', 
    'Min_Humidity_All_Years', 
    'Mean_Humidity_All_Years'
]




all_years_stats['city'] =  'SF'
all_years_stats

,Month,Max_Temperature_All_Years,Min_Temperature_All_Years,Mean_Temperature_All_Years,Max_Humidity_All_Years,Min_Humidity_All_Years,Mean_Humidity_All_Years,city
0,1,64.0,40.0,50.350358,99.0,33.0,79.065412,SF
1,2,64.0,38.0,52.851687,100.0,36.0,76.980655,SF
2,3,70.0,44.0,54.766577,98.0,33.0,74.558692,SF
3,4,76.0,46.0,56.631944,96.0,37.0,71.169444,SF
4,5,80.0,51.0,59.167563,92.0,40.0,70.972670,SF
5,6,85.0,54.0,61.833796,89.0,35.0,70.727778,SF
6,7,82.0,54.0,63.306452,93.0,35.0,73.503584,SF
7,8,82.0,54.0,64.012993,93.0,43.0,74.343190,SF
8,9,82.0,57.0,64.428704,94.0,31.0,72.602778,SF
9,10,81.0,52.0,61.968638,94.0,32.0,71.313620,SF


In [65]:

all_year_ave = pd.concat([all_year_ave, all_years_stats], ignore_index=True)
all_year_ave

,Month,Max_Temperature_All_Years,Min_Temperature_All_Years,Mean_Temperature_All_Years,Max_Humidity_All_Years,Min_Humidity_All_Years,Mean_Humidity_All_Years,city
0,1,80.0,41.0,68.103943,98.0,33.0,71.814516,Miami
1,2,82.0,47.0,70.008929,94.0,37.0,71.205853,Miami
2,3,84.0,42.0,72.315412,96.0,41.0,68.485215,Miami
3,4,86.0,57.0,75.813889,92.0,41.0,67.247222,Miami
4,5,86.0,67.0,79.546595,94.0,48.0,69.180556,Miami
5,6,90.0,70.0,82.356019,94.0,53.0,74.423611,Miami
6,7,90.0,75.0,83.676523,90.0,57.0,73.373656,Miami
7,8,89.0,76.0,83.886649,94.0,56.0,74.549283,Miami
8,9,88.0,74.0,82.605556,96.0,54.0,76.290741,Miami
9,10,87.0,64.0,79.484767,96.0,47.0,73.375448,Miami


In [66]:
all_year_ave.to_csv('yearlyAverage.csv', index=False)

DONE